In [1]:
import pandas as pd
import numpy as np

In [2]:
field = 'Liabilities'
field_indicator = 'liabilities'

In [3]:
df = pd.read_csv('liabilities_with_notes.csv')

In [4]:
del df['Unnamed: 0']

In [5]:
df.coreg = df.coreg.fillna('null')

In [6]:
len(df)

52690

In [7]:
# Drop duplicates 
df = df.drop_duplicates(['cik','coreg','period',field])
len(df)

52690

In [8]:
# Keeping the non-null values
df = df.copy()[df.copy()[field].notnull()]
len(df)

52690

## Keeping only the consolidated companies (coreg = null)

In [9]:
df_consolidated = df.copy()[df.copy().coreg=='null']

In [10]:
df_consolidated['cik'] = df_consolidated['cik'].astype('str')
df_consolidated['period'] = df_consolidated['period'].astype('str')

df_consolidated['cik+period'] = df_consolidated['cik'] + "_" + df_consolidated['period']

df_consolidated['cik'] = df_consolidated['cik'].astype('int64')
df_consolidated['period'] = df_consolidated['period'].astype('float64')

In [11]:
# List of cik+period pairs of companies that have consolidated(coreg=null) entries
df_consolidated_cik_and_period = df_consolidated['cik+period']

In [12]:
df_not_consolidated = df.copy()[df.copy().coreg!='null']

In [13]:
df_not_consolidated['cik'] = df_not_consolidated['cik'].astype('str')
df_not_consolidated['period'] = df_not_consolidated['period'].astype('str')

df_not_consolidated['cik+period'] = df_not_consolidated['cik'] + "_" + df_not_consolidated['period']

df_not_consolidated['cik'] = df_not_consolidated['cik'].astype('int64')
df_not_consolidated['period'] = df_not_consolidated['period'].astype('float64')

In [14]:
# List of companies that don't have consolidated company but only has subsidiaries
df_no_consolidated_in_group = df_not_consolidated[~df_not_consolidated.isin(df_consolidated_cik_and_period)]

In [15]:
# Summed up values into one for companies that don't have consolidated company but only has subsidiaries
df_no_consolidated_in_group_sum = \
df_no_consolidated_in_group.groupby(['cik','period']).sum().\
loc[:,[field]].reset_index().\
merge(df_no_consolidated_in_group.loc[:,['adsh', 'adsh+coreg+value','adsh_10k', 'adsh_other',
 'adsh_x', 'adsh_y','cik', 'cik+coreg+ddate',
 'cik+coreg+period','coreg', 'ddate',
 'filed', 'form','fp', 'fy',
 'fye','name','period', 'qtrs', 'sic','ein','cik+period']], on=['cik','period'],how='left').drop_duplicates()

In [16]:
# Fill in coreg of df_no_consolidated_in_group_sum as "sum"
df_no_consolidated_in_group_sum['coreg'] = 'sum'

In [17]:
got_only_consolidated_df =\
df_consolidated.append(df_no_consolidated_in_group_sum)

In [18]:
# 'cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field

got_only_consolidated_df = got_only_consolidated_df.loc[:,['cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field]]

In [19]:
got_only_consolidated_df = got_only_consolidated_df.drop_duplicates(['cik','period',field])

In [20]:
len(got_only_consolidated_df)

46855

In [21]:
# Number of unique cik, period combinations
got_only_consolidated_df.groupby(['cik','period']).ngroups

35700

## Duplicates(cik, coreg, period)

In [22]:
# List of duplicates(cik, coreg, period) from the list of consolidated companies
got_only_consolidated_df_dup =\
got_only_consolidated_df.copy()[got_only_consolidated_df.copy().duplicated(['cik','period'],keep=False)]

In [23]:
# List of entities that are not duplicates(cik, coreg, period) from the list of consolidated companies
got_only_consolidated_df_not_dup =\
got_only_consolidated_df.copy().drop_duplicates(['cik','period'],keep=False)

In [24]:
# Number of unique cik, coreg, period combinations that have duplicates
got_only_consolidated_df_dup.groupby(['cik','period']).ngroups

5699

In [25]:
got_only_consolidated_df_dup['qtrs_check'] = \
np.where(got_only_consolidated_df_dup.qtrs==4.0,100,-1)

In [26]:
#### STATS ####
# Number of entities that have just one qtrs=4
sum((pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check <=100) & \
    (pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check >0))

0

In [27]:
#### STATS ####
# Number of entities that have multiple qtrs=4
sum(pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check>100)

0

In [28]:
#### STATS ####
# Number of entities that don't have any qtrs=4 at all
sum(pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check<0)

5699

In [29]:
#### STATS ####
# Number of entities that we have to check manually
sum(pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check>100) + \
sum(pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','period']).qtrs_check.sum()).reset_index().qtrs_check<0)

5699

## Make data_duplicates & Clean

In [30]:
# Dataframe that shows for each cik+coreg+period c|ombination how many qtrs=4 it has
df_dup_qtrs4_check =\
pd.DataFrame(got_only_consolidated_df_dup.groupby(['cik','coreg','period']).qtrs_check.sum()).reset_index()

In [31]:
# 'cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field

# List of entities that don't have any qtrs=4 at all
df_dup_no_qtrs4_at_all = \
df_dup_qtrs4_check[df_dup_qtrs4_check.qtrs_check<0].\
merge(got_only_consolidated_df_dup.loc[:,['cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field]], 
      on=['cik','period'],how='left')

In [32]:
# 'cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field

# List of entities that don't have any qtrs=4 at all
df_dup_multip_qtrs4= \
df_dup_qtrs4_check[df_dup_qtrs4_check.qtrs_check>100].\
merge(got_only_consolidated_df_dup.loc[:,['cik', 'name','sic', 'period', 'fy', 'qtrs', 'ein', field]], 
      on=['cik','period'],how='left')

In [33]:
# List of entities that we have check manually
df_dup_manually_check =\
df_dup_no_qtrs4_at_all.append(df_dup_multip_qtrs4)

del df_dup_manually_check['qtrs_check']

In [34]:
# Saving duplicate dataframe that we need to manually check as field_duplicates.csv
df_dup_manually_check.to_csv(field_indicator+'_duplicates.csv')

In [35]:
# 'cik', 'name', 'coreg', 'sic', 'period', 'fy', 'qtrs', 'ein', field

# List of entities tht only have one qtrs=4
df_dup_just_one_qtrs4= \
df_dup_qtrs4_check[(df_dup_qtrs4_check.qtrs_check<=100) & (df_dup_qtrs4_check.qtrs_check>0)].\
merge(got_only_consolidated_df_dup.loc[:,['cik', 'name','sic', 'period', 'fy', 'qtrs', 'ein', field]],
      on=['cik','period'],how='left')

del df_dup_just_one_qtrs4['qtrs_check']

# Keep only the entities tha have qtrs=4
df_dup_just_one_qtrs4 = df_dup_just_one_qtrs4.copy()[df_dup_just_one_qtrs4.copy().qtrs==4.0]

In [36]:
# Clean Data (Non duplicates + df that has only one qtrs=4)
df_clean_data =\
got_only_consolidated_df_not_dup.append(df_dup_just_one_qtrs4)

In [37]:
len(df_clean_data), df_clean_data.groupby(['cik','period']).ngroups

(30001, 30001)

In [38]:
# Finally drop duplicates that still have the same cik, period and value
df_clean_data.drop_duplicates(['cik','period',field],inplace=True)

In [39]:
df_clean_data[df_clean_data.duplicated(['cik','period'],keep=False)]

,Liabilities,cik,coreg,ein,fy,name,period,qtrs,sic


In [87]:
# If there are still duplicates even after all this, keep last duplicates that still have the same cik, period and value 
df_clean_data.drop_duplicates(['cik','period'],keep='last',inplace=True)

In [88]:
df_clean_data[df_clean_data.duplicated(['cik','period'],keep=False)]

,ResearchAndDevelopmentExpense,cik,coreg,ein,fy,name,period,qtrs,sic


In [40]:
# Saving cleaned data as "field indicator name_clean.csv"
df_clean_data.to_csv(field_indicator+'_clean.csv')